In [ ]:
import pandas as pd
from scipy import stats

In [ ]:
# Чтение файла CSV
# df = pd.read_excel('statistics.xlsm')
df = pd.read_csv('data/statistics.scv')
df.dropna(thresh=50,inplace=True)
df.dropna(thresh=50,axis=1,inplace=True)
df.drop(['№','ФИО','номер карты'], axis=1,inplace=True)
print(df.shape)
df['код диагноза']= df['код диагноза'].str.replace('C56','1')
df['код диагноза']= df['код диагноза'].str.replace('D27','0')
df['код диагноза']= df['код диагноза'].astype(int)
# удаляем из df все строки, содержащие NaN в каждой из колонок

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
group_column = 'код диагноза'
groups = df[group_column].unique()



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Разделение на признаки и целевую переменную
X = df.drop('код диагноза', axis=1)  # Признаки
y = df['код диагноза']  # Целевая переменная (группа 0 или 1)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Стандартизация данных (LASSO чувствителен к масштабу данных)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение модели LASSO (логистическая регрессия с L1-регуляризацией)
lasso_model = LogisticRegression(penalty='l1', solver='liblinear', C=0.3, random_state=42)
lasso_model.fit(X_train_scaled, y_train)

# Предсказание на тестовой выборке
y_pred = lasso_model.predict(X_test_scaled)

In [ ]:
# Оценка модели
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, lasso_model.predict_proba(X_test_scaled)[:, 1])
print(f'Accuracy: {accuracy:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')

In [ ]:
# Матрица ошибок
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Group 0', 'Group 1'], yticklabels=['Group 0', 'Group 1'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Вывод коэффициентов LASSO
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lasso_model.coef_[0]
})
coef_df = coef_df.sort_values(by='Coefficient', key=abs, ascending=False)

# Отбор наиболее важных признаков (ненулевые коэффициенты)
important_features = coef_df[coef_df['Coefficient'] != 0]
print("Наиболее важные признаки:")
print(important_features)

In [ ]:
# Визуализация коэффициентов
plt.figure(figsize=(10, 6))
plt.barh(important_features['Feature'], important_features['Coefficient'])
plt.xlabel('Коэффициент LASSO')
plt.ylabel('Признак')
plt.title('Важность признаков (LASSO коэффициенты)')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели случайного леса
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка модели
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
print(f'Accuracy: {accuracy}, ROC-AUC: {roc_auc}')

In [ ]:
# Матрица ошибок
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# Важность признаков
importances = model.feature_importances_
feature_importances = pd.Series(importances, index=X.columns)
feature_importances.sort_values(ascending=False).plot(kind='bar')
plt.title('Feature Importances')
plt.show()